<a href="https://colab.research.google.com/github/Ikuesan/Projects/blob/main/Student_score_prediction_with_Neural%20Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
df = pd.read_csv("Student_score_data.csv")

df

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,816,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65
30637,890,male,group E,high school,standard,none,single,regularly,no,1.0,private,5 - 10,58,53,51
30638,911,female,NaN,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67
30639,934,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93


In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
Gender,0
EthnicGroup,1840
ParentEduc,1845
LunchType,0
TestPrep,1830
ParentMaritalStatus,1190
PracticeSport,631
IsFirstChild,904
NrSiblings,1572


In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
for col in df.columns:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(dropna=False))


Column: Unnamed: 0
Unnamed: 0
828    31
0      31
1      31
2      31
802    31
       ..
173    28
477    28
417    28
299    28
106    28
Name: count, Length: 1000, dtype: int64

Column: Gender
Gender
female    15424
male      15217
Name: count, dtype: int64

Column: EthnicGroup
EthnicGroup
group C    9212
group D    7503
group B    5826
group E    4041
group A    2219
NaN        1840
Name: count, dtype: int64

Column: ParentEduc
ParentEduc
some college          6633
high school           5687
associate's degree    5550
some high school      5517
bachelor's degree     3386
master's degree       2023
NaN                   1845
Name: count, dtype: int64

Column: LunchType
LunchType
standard        19905
free/reduced    10736
Name: count, dtype: int64

Column: TestPrep
TestPrep
none         18856
completed     9955
NaN           1830
Name: count, dtype: int64

Column: ParentMaritalStatus
ParentMaritalStatus
married     16844
single       7097
divorced     4919
NaN          1190
widowed

In [ ]:
#drop unecessary column
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65
30637,male,group E,high school,standard,none,single,regularly,no,1.0,private,5 - 10,58,53,51
30638,female,NaN,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67
30639,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93


In [ ]:
#filling the missing values of categorical columns with mode
categorical_cols = [
    'EthnicGroup', 'ParentEduc', 'TestPrep', 'ParentMaritalStatus',
    'PracticeSport', 'IsFirstChild', 'WklyStudyHours', 'TransportMeans'
]

for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

df

<ipython-input-87-f855839f462a>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,group C,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,female,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,69,90,88
2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,school_bus,5 - 10,45,56,42
4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65
30637,male,group E,high school,standard,none,single,regularly,no,1.0,private,5 - 10,58,53,51
30638,female,group C,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67
30639,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93


In [ ]:
df.isnull().sum()

,0
Gender,0
EthnicGroup,0
ParentEduc,0
LunchType,0
TestPrep,0
ParentMaritalStatus,0
PracticeSport,0
IsFirstChild,0
NrSiblings,1572
TransportMeans,0


In [ ]:
#filling missing values of numerical column
df['NrSiblings'].fillna(df['NrSiblings'].median(), inplace=True)

df

<ipython-input-89-5976b2c629c0>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['NrSiblings'].fillna(df['NrSiblings'].median(), inplace=True)


,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,group C,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,female,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,69,90,88
2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,school_bus,5 - 10,45,56,42
4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,5 - 10,59,61,65
30637,male,group E,high school,standard,none,single,regularly,no,1.0,private,5 - 10,58,53,51
30638,female,group C,high school,free/reduced,completed,married,sometimes,no,1.0,private,5 - 10,61,70,67
30639,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,5 - 10,82,90,93


In [ ]:
df.isnull().sum()

,0
Gender,0
EthnicGroup,0
ParentEduc,0
LunchType,0
TestPrep,0
ParentMaritalStatus,0
PracticeSport,0
IsFirstChild,0
NrSiblings,0
TransportMeans,0


In [ ]:
for col in df.columns:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(dropna=False))


Column: Gender
Gender
female    15424
male      15217
Name: count, dtype: int64

Column: EthnicGroup
EthnicGroup
group C    11052
group D     7503
group B     5826
group E     4041
group A     2219
Name: count, dtype: int64

Column: ParentEduc
ParentEduc
some college          8478
high school           5687
associate's degree    5550
some high school      5517
bachelor's degree     3386
master's degree       2023
Name: count, dtype: int64

Column: LunchType
LunchType
standard        19905
free/reduced    10736
Name: count, dtype: int64

Column: TestPrep
TestPrep
none         20686
completed     9955
Name: count, dtype: int64

Column: ParentMaritalStatus
ParentMaritalStatus
married     18034
single       7097
divorced     4919
widowed       591
Name: count, dtype: int64

Column: PracticeSport
PracticeSport
sometimes    15844
regularly    10793
never         4004
Name: count, dtype: int64

Column: IsFirstChild
IsFirstChild
yes    19986
no     10655
Name: count, dtype: int64

Column: NrS

In [ ]:
df["ParentEduc"].unique()

array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object)

In [ ]:
#Ordinal encoding for the ordinal categorical columns

educ_order = {
   'some high school': 0,
    'high school': 1,
    'some college': 2,
    "associate's degree": 3,
    "bachelor's degree": 4,
    "master's degree": 5
}
df['ParentEduc'] = df['ParentEduc'].map(educ_order)


In [ ]:
df["WklyStudyHours"].unique()

array(['< 5', '5 - 10', '> 10'], dtype=object)

In [ ]:
study_order = {
    '< 5': 0,
    '5 - 10': 1,
    '> 10': 2,
}
df["WklyStudyHours"] = df["WklyStudyHours"].map(study_order)


In [ ]:
#OneHot Encoding for the nominal features
nominal_features = [
    'EthnicGroup',
    'ParentMaritalStatus',
    'PracticeSport',
]

In [ ]:
# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform
encoded_array = encoder.fit_transform(df[nominal_features])

# Convert to a dense array if needed (optional, as a sparse matrix might be more memory efficient)
encoded_array = encoded_array.toarray()

# Convert to DataFrame with new column names
encoded_df = pd.DataFrame(
    encoded_array,
    columns=encoder.get_feature_names_out(nominal_features)
)

In [ ]:
encoded_df

,EthnicGroup_group A,EthnicGroup_group B,EthnicGroup_group C,EthnicGroup_group D,EthnicGroup_group E,ParentMaritalStatus_divorced,ParentMaritalStatus_married,ParentMaritalStatus_single,ParentMaritalStatus_widowed,PracticeSport_never,PracticeSport_regularly,PracticeSport_sometimes
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30636,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
30637,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
30638,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
30639,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Drop the original nominal columns from the main df
df_dropped = df.drop(columns=nominal_features)

# Concatenate the encoded columns
df_encoded = pd.concat([encoded_df, df_dropped], axis=1)

In [ ]:
df_encoded

,EthnicGroup_group A,EthnicGroup_group B,EthnicGroup_group C,EthnicGroup_group D,EthnicGroup_group E,ParentMaritalStatus_divorced,ParentMaritalStatus_married,ParentMaritalStatus_single,ParentMaritalStatus_widowed,PracticeSport_never,...,ParentEduc,LunchType,TestPrep,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4,standard,none,yes,3.0,school_bus,0,71,71,74
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2,standard,none,yes,0.0,school_bus,1,69,90,88
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5,standard,none,yes,4.0,school_bus,0,87,93,91
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,3,free/reduced,none,no,1.0,school_bus,1,45,56,42
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2,standard,none,yes,0.0,school_bus,1,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,standard,none,no,2.0,school_bus,1,59,61,65
30637,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1,standard,none,no,1.0,private,1,58,53,51
30638,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,free/reduced,completed,no,1.0,private,1,61,70,67
30639,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3,standard,completed,no,3.0,school_bus,1,82,90,93


In [ ]:
from sklearn.preprocessing import LabelEncoder

df_encoded_ = df_encoded.copy()

binary_features = ['Gender', 'TestPrep', 'IsFirstChild', 'TransportMeans', 'LunchType']

label_encoder = LabelEncoder()

for col in binary_features:
    df_encoded_[col] = label_encoder.fit_transform(df_encoded_[col])


In [ ]:
df_encoded_

,EthnicGroup_group A,EthnicGroup_group B,EthnicGroup_group C,EthnicGroup_group D,EthnicGroup_group E,ParentMaritalStatus_divorced,ParentMaritalStatus_married,ParentMaritalStatus_single,ParentMaritalStatus_widowed,PracticeSport_never,...,ParentEduc,LunchType,TestPrep,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,4,1,1,1,3.0,1,0,71,71,74
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2,1,1,1,0.0,1,1,69,90,88
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,5,1,1,1,4.0,1,0,87,93,91
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,3,0,1,0,1.0,1,1,45,56,42
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2,1,1,1,0.0,1,1,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30636,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,1,1,0,2.0,1,1,59,61,65
30637,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1,1,1,0,1.0,0,1,58,53,51
30638,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,0,0,0,1.0,0,1,61,70,67
30639,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3,1,0,0,3.0,1,1,82,90,93


In [ ]:
for col in df_encoded_.columns:
    print(f"\nColumn: {col}")
    print(df_encoded_[col].value_counts(dropna=False))


Column: EthnicGroup_group A
EthnicGroup_group A
0.0    28422
1.0     2219
Name: count, dtype: int64

Column: EthnicGroup_group B
EthnicGroup_group B
0.0    24815
1.0     5826
Name: count, dtype: int64

Column: EthnicGroup_group C
EthnicGroup_group C
0.0    19589
1.0    11052
Name: count, dtype: int64

Column: EthnicGroup_group D
EthnicGroup_group D
0.0    23138
1.0     7503
Name: count, dtype: int64

Column: EthnicGroup_group E
EthnicGroup_group E
0.0    26600
1.0     4041
Name: count, dtype: int64

Column: ParentMaritalStatus_divorced
ParentMaritalStatus_divorced
0.0    25722
1.0     4919
Name: count, dtype: int64

Column: ParentMaritalStatus_married
ParentMaritalStatus_married
1.0    18034
0.0    12607
Name: count, dtype: int64

Column: ParentMaritalStatus_single
ParentMaritalStatus_single
0.0    23544
1.0     7097
Name: count, dtype: int64

Column: ParentMaritalStatus_widowed
ParentMaritalStatus_widowed
0.0    30050
1.0      591
Name: count, dtype: int64

Column: PracticeSport_neve

In [ ]:
#model building

X = df_encoded_.drop(columns=['WritingScore', 'MathScore','ReadingScore'])
y = df_encoded_[['WritingScore', 'MathScore','ReadingScore']]

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Scale inputs
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='linear')  # Output layer has 3 neurons (one per score)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,523 (13.76 KB)

 Trainable params: 3,523 (13.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 220.5513 - mae: 11.7469 - val_loss: 179.5353 - val_mae: 10.8830
Epoch 2/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 161.4807 - mae: 10.2764 - val_loss: 178.3410 - val_mae: 10.8238
Epoch 3/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 158.8225 - mae: 10.2106 - val_loss: 178.3190 - val_mae: 10.8491
Epoch 4/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 157.2822 - mae: 10.1212 - val_loss: 177.5835 - val_mae: 10.8017
Epoch 5/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 158.6487 - mae: 10.1854 - val_loss: 176.6483 - val_mae: 10.8008
Epoch 6/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 159.8016 - mae: 10.2028 - val_loss: 176.7881 - val_mae: 10.8065
Epoch 7/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 157.5730 - mae: 10.1489 - val_loss: 179.8060 - val_mae: 10.8478
Epoch 8/100
613/613 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 158.1336 - mae: 10.1420 - val_loss: 177.3124 - val_mae: 10.8250


In [ ]:
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.2f}, Test Mean Absolute Error: {mae:.2f}")

192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 183.0251 - mae: 10.9861
Test Loss: 182.11, Test Mean Absolute Error: 10.91


In [ ]:
predictions = model.predict(X_test_scaled)

192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
comparison = pd.DataFrame(predictions, columns=['Pred_Math', 'Pred_Reading', 'Pred_Writing'])
comparison[['True_Math', 'True_Reading', 'True_Writing']] = y_test.reset_index(drop=True)
comparison.head()

,Pred_Math,Pred_Reading,Pred_Writing,True_Math,True_Reading,True_Writing
0,76.590607,66.333832,75.806030,87,70,87
1,78.207573,86.512177,78.681831,84,90,83
2,61.070400,67.237450,64.453903,81,81,89
3,66.773697,64.750244,69.547035,55,55,57
4,61.152061,60.614174,63.831894,53,49,50


In [ ]:
#